<a href="https://colab.research.google.com/github/MohammadAghaei1/Generative-AI/blob/main/GANs_%26_VAEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time

# **Loading dataset**

In [2]:
# LOAD AND PREPROCESS DATA
print("Loading MNIST data...")
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Reshape from (60000, 28, 28) → (60000, 28, 28, 1)
# Add channel dimension for Conv2D (1 = grayscale)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

# Normalize pixel values from [0, 255] → [-1, 1]
train_images = (train_images - 127.5) / 127.5

# Dataset parameters
BUFFER_SIZE = 60000
BATCH_SIZE = 128

#  - shuffle it
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE)

# Check the shape of the data
print(train_dataset.shape)

**Checking activation of GPU**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


# **Making Generator**

In [ ]:
def make_generator_model(noise_dim):
    model = tf.keras.Sequential()

    # Input: Random Noise
    # # Start with a dense layer that outputs a 7x7x128 tensor
    model.add(layers.Dense(7*7*128, use_bias=False, input_shape=(noise_dim,)))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((7, 7, 128)))

    # Upsample from 7x7 to 14x14
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU())

    # Upsample from 14x14 to 28x28
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization(momentum=0.8))  #
    model.add(layers.LeakyReLU())

    # Refine 28x28 (Stride = 1)
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))

    return model

'def build_generator():\n\n    model = Sequential()\n\n\n    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=100))\n    model.add(Reshape((7, 7, 128)))\n\n    model.add(UpSampling2D())\n    model.add(Conv2D(128, kernel_size=3, padding="same"))\n    model.add(BatchNormalization(momentum=0.8))\n    model.add(Activation("relu"))\n\n    model.add(UpSampling2D())\n    model.add(Conv2D(64, kernel_size=3, padding="same"))\n    model.add(BatchNormalization(momentum=0.8))\n    model.add(Activation("relu"))\n\n    model.add(Conv2D(1, kernel_size=3, padding="same"))\n    model.add(Activation("tanh"))\n\n    model.summary()\n\n    noise = Input(shape=(100,))\n    img = model(noise)\n\n    return Model(noise, img)'

In [ ]:
NOISE_DIM = 100
generator = make_generator_model(NOISE_DIM)

# **Making Discriminator**

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()

    # --- Block 1: 32 Filters (Input) ---
    # Matches Code A: Conv2D(32) -> LeakyReLU -> Dropout
    model.add(layers.Conv2D(32, kernel_size=3, strides=2, padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # --- Block 2: 64 Filters ---
    # Matches Code A: Conv2D(64) -> BN -> LeakyReLU -> Dropout
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same'))
    # Note: I used padding='same' instead of manual ZeroPadding2D to prevent shape errors
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # --- Block 3: 128 Filters ---
    # Matches Code A: Conv2D(128) -> BN -> LeakyReLU -> Dropout
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # --- Block 4: 256 Filters ---
    # Matches Code A: Conv2D(256) -> BN -> LeakyReLU -> Dropout
    # Note: Strides=1 here (just like Code A) to refine features without shrinking size
    model.add(layers.Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # --- Output ---
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

'def build_discriminator():\n\n    model = Sequential()\n\n    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(28,28,1), padding="same"))\n    model.add(LeakyReLU(alpha=0.2))\n    model.add(Dropout(0.25))\n\n    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))\n    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))\n    model.add(BatchNormalization(momentum=0.8))\n    model.add(LeakyReLU(alpha=0.2))\n    model.add(Dropout(0.25))\n\n    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))\n    model.add(BatchNormalization(momentum=0.8))\n    model.add(LeakyReLU(alpha=0.2))\n    model.add(Dropout(0.25))\n\n    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))\n    model.add(BatchNormalization(momentum=0.8))\n    model.add(LeakyReLU(alpha=0.2))\n    model.add(Dropout(0.25))\n\n    model.add(Flatten())\n    model.add(Dense(1, activation=\'sigmoid\'))\n\n    model.summary()\n\n    img = Input(shape=(28,28,1))\n    validity = model(img)\n

In [ ]:
discriminator = make_discriminator_model()

# **Loss functions and optimizers**

In [ ]:
# Binary cross-entropy loss for real/fake classification
# from_logits=False because discriminator uses sigmoid
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    """
    real_output: D(real images)  -> should be close to 1
    fake_output: D(fake images)  -> should be close to 0
    """
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)   # label 1 for real
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)  # label 0 for fake
    return real_loss + fake_loss

def generator_loss(fake_output):
    """
    fake_output: D(fake images)
    Generator wants D(fake) ≈ 1 (fool the discriminator)
    """
    return cross_entropy(tf.ones_like(fake_output), fake_output)  # want label 1 for fake


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)

# **Making DCGAN**

In [ ]:
@tf.function
def train_step(images):
    """
    Performs one training step on a batch of real images:
      - sample noise
      - generate fake images
      - compute discriminator and generator losses
      - update both networks
      - compute discriminator accuracy on real and fake
    """
    # Sample random noise for the generator: (batch_size, NOISE_DIM)
    # Here batch_size is fixed to 128, matching BATCH_SIZE
    noise = tf.random.normal([128, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images from noise
        generated_images = generator(noise, training=True)

        # Discriminator output for real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Compute losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Compute gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply gradients (update weights)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # ----- Discriminator accuracies -----
    # Real images: correct if prediction > 0.5 (classified as real)
    real_pred = tf.cast(real_output > 0.5, tf.float32)
    real_acc = tf.reduce_mean(real_pred)  # fraction of real images correctly classified

    # Fake images: correct if prediction <= 0.5 (classified as fake)
    fake_pred_real = tf.cast(fake_output > 0.5, tf.float32)  # 1 if predicted real (wrong)
    fake_acc = tf.reduce_mean(1.0 - fake_pred_real)          # 1 - wrong = correct

    return gen_loss, disc_loss, real_acc, fake_acc


**Function for saving and plotting images**

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    """
    Generate images from a fixed noise vector (test_input)
    and plot them in a 4x4 grid.
    """
    # Disable training behavior (e.g., batchnorm updates)
    predictions = model(test_input, training=False)

    plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)

        # Convert from [-1, 1] back to [0, 1] for display
        img_to_plot = (predictions[i, :, :, 0] + 1) / 2.0

        plt.imshow(img_to_plot, cmap='gray')
        plt.axis('off')

    plt.suptitle(f'Epoch {epoch}')
    plt.show()

# **Traning**

In [ ]:
def train(dataset, epochs):
    print("Starting training with Tanh [-1, 1]...")

    # Batch the dataset here (we kept it unbatched before)
    BATCH_SIZE = 128
    dataset = dataset.batch(BATCH_SIZE)

    for epoch in range(epochs):
        g_loss_metric = 0.0
        d_loss_metric = 0.0
        real_acc_metric = 0.0
        fake_acc_metric = 0.0
        steps = 0

        # Iterate over all batches in the dataset
        for image_batch in dataset:
            g_loss, d_loss, real_acc, fake_acc = train_step(image_batch)
            g_loss_metric += g_loss
            d_loss_metric += d_loss
            real_acc_metric += real_acc
            fake_acc_metric += fake_acc
            steps += 1

        # Average over all batches (epoch metrics)
        avg_g_loss = g_loss_metric / steps
        avg_d_loss = d_loss_metric / steps
        avg_real_acc = real_acc_metric / steps
        avg_fake_acc = fake_acc_metric / steps

        # Print losses + discriminator accuracies (in %)
        print(
            f'Epoch {epoch + 1}, '
            f'Gen Loss: {avg_g_loss:.4f}, '
            f'Disc Loss: {avg_d_loss:.4f}, '
            f'D(real)%: {avg_real_acc * 100:.1f}, '
            f'D(fake)%: {avg_fake_acc * 100:.1f}'
        )

        # Visualize 16 generated images from a fixed noise vector
        generate_and_save_images(generator, epoch + 1, seed)


**Running DCGAN**

In [ ]:
# Fixed noise vector used for monitoring generator progress
seed = tf.random.normal([16, NOISE_DIM])
train(train_dataset, epochs=50)